##1.2 Variable 클래스 구현

In [1]:
class Variable:
  def __init__(self, data):
    self.data = data

In [2]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [3]:
x.data = np.array(2.0)
print(x.data)

2.0


##1.3 넘파이의 다차원 배열

In [4]:
import numpy as np
x = np.array(1)
x.ndim

0

In [5]:
x = np.array([1,2,3])
x.ndim

1

In [6]:
x = np.array([[1,2,3],
              [4,5,6]])
x.ndim

2

##2.2 Fucntion 클래스 구현

In [7]:
class Function:
  def __call__(self, input):
    x = input.data
    y = x**2
    output = Variable(y)
    return output

##2.3 Function 클래스 이용

In [8]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [9]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output

  def forward(self,x):
    raise NotImplementedError()

In [10]:
class Square(Function):
  def forward(self,x):
    return x**2

In [11]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


##3.1 Exp 함수 구현

In [12]:
class Exp(Function):
  def forward(self,x):
    return np.exp(x)

##3.2 함수 연결

In [13]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


##4.2 수치 미분 구현

In [14]:
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data-eps)
  x1 = Variable(x.data+eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2*eps)

In [15]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f,x)
print(dy)

4.000000000004


##4.3 합성 함수의 미분

In [16]:
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f,x)
print(dy)

3.2974426293330694


##6.1 Variable 클래스 추가 구현

In [17]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None

##6.2 Function 클래스 추가 구현

In [18]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    self.input = input
    return output

  def forward(self,x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

##6.3 Squre와 Exp 클래스 추가 구현

In [19]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y

  def backward(self, gy):
    x = self.input.data
    gx = 2*x*gy
    return gx

In [20]:
class Exp(Function):
  def forward(self,x):
    y = np.exp(x)
    return y

  def backward(self,gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

##6.4 역전파 구현

In [21]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [22]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


##7.1 역전파 자동화의 시작

In [23]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self,func):
    self.creator = func

In [28]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

##7.3 backward 메서드 추가

In [27]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    f = self.creator
    if f is not None:
      x = f.input
      x.grad = f.backward(self.grad)
      x.backward()

In [29]:
## ?????왜 계속 None으로 뜰까요.....
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

None


##8.2 반복문을 이용한 구현

In [30]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

##9.1 파이썬 함수로 이용하기

In [32]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

In [ ]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

In [ ]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print(x.grad)

##9.2 backward 메서드 간소화

In [36]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)

##9.3 ndarray만 취급하기

In [38]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)

In [39]:
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

In [40]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(as_array(y))
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

##10.1 파이썬 단위 테스트

In [41]:
import unittest
class SquareTest(unittest.TestCase):
  def test_forward(self):
    x = Variable(np.array(2.0))
    y = square(x)
    expected = np.array(4.0)
    self.assertEqual(y.data, expected)

##10.2 square 함수의 역전파 테스트

In [43]:
class SquareTest(unittest.TestCase):
  def test_forward(self):
    x = Variable(np.array(2.0))
    y = square(x)
    expected = np.array(4.0)
    self.assertEqual(y.data, expected)

  def test_backward(self):
    x = Variable(np.array(3.0))
    y = square(x)
    y.backward()
    expected = np.array(6.0)
    self.assertEqual(x.grad, expected)

##10.3 기울기 확인을 이용한 자동 테스트

In [44]:
def numeric_diff(f, x, eps=1e-4):
  x0 = Variable(x.data-eps)
  x1 = Variable(x.data+eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data-y0.data)/(2*eps)

class SquareTest(unittest.TestCase):
  def test_forward(self):
    x = Variable(np.array(2.0))
    y = square(x)
    expected = np.array(4.0)
    self.assertEqual(y.data, expected)

  def test_backward(self):
    x = Variable(np.array(3.0))
    y = square(x)
    y.backward()
    expected = np.array(6.0)
    self.assertEqual(x.grad, expected)

  def test_gradient_check(self):
    x = Variable(np.random.rand(1))
    y = square(x)
    y.backward()
    num_grad = numerical_diff(square,x)
    flg = np.allclose(x.grad, num_grad)
    self.assertTrue(flg)